In [3]:
from tqdm import tqdm

import pandas as pd
import numpy as np

from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import DataFrameLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [4]:
class CFG:
    # store="프랭크버거"
    output_path = "/home/user09/beaver/data/db"
    save_path = ""
    embedding_model="BAAI/bge-m3"
    retriever_k=5
    retriever_bert_weight=0.7
    version='10'
    seed=42
    
# CFG.save_path = CFG.output_path + CFG.store + "_temp.csv"

In [5]:
#### 엑셀파일 DB화(pickle파일로 변환) ####
store = "홍콩반점"

from langchain.schema import Document

# df_2 = pd.read_excel(f'/home/user09/beaver/data/dataset_v{CFG.version}.xlsx')
# loader = DataFrameLoader(df_2, page_content_column="상품명")
# docs = loader.load()
df_2 = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}.xlsx')

# 상품명, 종류, 옵션, 가격을 하나의 문서로 결합하여 저장
docs = [
    Document(
        page_content=row['상품명'],
        metadata={
            '옵션': row['옵션'],
            '특징': row['특징']
        }
    )
    for _, row in df_2.iterrows()
]   
    
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

encode_kwargs={'normalize_embeddings':True}
model_kwargs={'device':'cpu'}

hf = HuggingFaceBgeEmbeddings(
    model_name=CFG.embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

db = FAISS.from_documents(docs, hf)

db.save_local(f"{CFG.output_path}/{store}_faiss{CFG.version}")
import pickle 
with open(f"{CFG.output_path}/{store}_docs{CFG.version}.pkl", "wb") as f:
    pickle.dump(docs, f)
    
db = FAISS.load_local(f"/home/user09/beaver/data/db/{store}_faiss{CFG.version}", hf, allow_dangerous_deserialization=True)
    
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": CFG.retriever_k}
)

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = CFG.retriever_k

ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[CFG.retriever_bert_weight, 1 - CFG.retriever_bert_weight],
)

KeyError: '상품명'